In [1]:
import pandas as pd

In [2]:
table_url = 'https://www.mapsofworld.com/thematic-maps/types-of-governments.html'
tables = pd.read_html(table_url)
tables

[            0
 0  Disclaimer,
                                                    0  \
 0  CloseDisclaimer : All efforts have been made t...   
 
                                                    1  
 0  CloseDisclaimer : All efforts have been made t...  ,
                                                    0  \
 0  ColorGovernment Type Transitional government F...   
 
                                                    1  
 0  ColorGovernment Type Communist state Democrati...  ,
     Color                                    Government Type
 0     NaN                            Transitional government
 1     NaN                                         Federation
 2     NaN                        Federal democratic Republic
 3     NaN     Constitutional monarchy and commonwealth realm
 4     NaN  Federal Parliamentary democracy and a Commonwe...
 5     NaN                 Democratic Constitutional Republic
 6     NaN                             Constitutional emirate
 7     NaN     

In [3]:
gov_df = tables[5]
gov_df.columns = ['Country', 'Govt Type', 'Head of State', 'Head of Government']
new_gov_df = gov_df.rename(columns={'Country':'country','Govt Type':'govt_type', 'Head of State':'head_of_state', 'Head of Government':'head_of_government'})
new_gov_df.set_index('country', inplace=True)
new_gov_df

,govt_type,head_of_state,head_of_government
country,,,
Afghanistan,Presidential Islamic republic,President - Ashraf Ghani,Chief Executive Officer - Abdullah Abdullah
Albania,Parliamentary republic,President - Bujar Nishani,Prime Minister - Edi Rama
Algeria,Presidential republic,President - Abdelaziz Bouteflika,Prime Minister - Abdelmalek Sellal
Andorra,Unitary parliamentary Semi-Elective diarchy,Episcopal Co-Prince - Joan Enric Vives Sicília...,Head of Government - Antoni Martí
Angola,Presidential republic,President - Jose Eduardo dos Santos,NaN
...,...,...,...
Venezuela,Federal presidential republic,President - Nicolas Maduro,NaN
Vietnam,Communist state,President - Tran Dại Quang,Prime Minister - Nguyen Xuan Phúc
Yemen,in transition (Provisional government),President - Abdrabbuh Mansur Hadi,Prime Minister - Ahmed Obeid bin Daghr


In [4]:
# The path to our CSV file
file = "Resources/happy_2019.csv"

hpi_country_df = pd.read_csv(file, encoding='UTF-8')
new_hpi_country_df = hpi_country_df.rename(columns={'Country or region':'country'})

hpi_final = new_hpi_country_df['country']
hpi_final.head(155)

0      Afghanistan
1          Albania
2          Algeria
3        Argentina
4          Armenia
          ...     
150     Uzbekistan
151      Venezuela
152        Vietnam
153          Yemen
154         Zambia
Name: country, Length: 155, dtype: object

In [5]:
merge_df = pd.merge(new_gov_df, hpi_final, on=('country'), how='inner')

clean_df = merge_df.rename(columns={ 'sub_region':'region'})

clean_df

,country,govt_type,head_of_state,head_of_government
0,Afghanistan,Presidential Islamic republic,President - Ashraf Ghani,Chief Executive Officer - Abdullah Abdullah
1,Albania,Parliamentary republic,President - Bujar Nishani,Prime Minister - Edi Rama
2,Algeria,Presidential republic,President - Abdelaziz Bouteflika,Prime Minister - Abdelmalek Sellal
3,Argentina,Presidential republic,President - Mauricio Macri,NaN
4,Armenia,Semi-presidential republic,President - Serzh Sargsyan,Prime Minister - Karen Karapetyan
...,...,...,...,...
141,Venezuela,Federal presidential republic,President - Nicolas Maduro,NaN
142,Vietnam,Communist state,President - Tran Dại Quang,Prime Minister - Nguyen Xuan Phúc
143,Yemen,in transition (Provisional government),President - Abdrabbuh Mansur Hadi,Prime Minister - Ahmed Obeid bin Daghr
144,Zambia,Presidential republic,President - Edgar Lungu,NaN


In [6]:
from sqlalchemy import create_engine
from config import postgres_pwd

db_name = "happiness_db"
table_name = "countries_by_government"

connection_string = f"postgres:{postgres_pwd}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')
conn = engine.connect()

clean_df.to_sql(table_name, con=engine, if_exists="append", index=False)

conn.close()

IntegrityError: (psycopg2.errors.ForeignKeyViolation) insert or update on table "countries_by_government" violates foreign key constraint "fk_countries_by_government_country"
DETAIL:  Key (country)=(United Kingdom) is not present in table "countries".

[SQL: INSERT INTO countries_by_government (country, govt_type, head_of_state, head_of_government) VALUES (%(country)s, %(govt_type)s, %(head_of_state)s, %(head_of_government)s)]
[parameters: ({'country': 'Afghanistan', 'govt_type': 'Presidential Islamic republic', 'head_of_state': 'President\xa0- Ashraf Ghani', 'head_of_government': 'Chief Executive Officer\xa0- Abdullah Abdullah'}, {'country': 'Albania', 'govt_type': 'Parliamentary republic', 'head_of_state': 'President\xa0- Bujar Nishani', 'head_of_government': 'Prime Minister\xa0- Edi Rama'}, {'country': 'Algeria', 'govt_type': 'Presidential republic', 'head_of_state': 'President\xa0- Abdelaziz Bouteflika', 'head_of_government': 'Prime Minister\xa0- Abdelmalek Sellal'}, {'country': 'A